# GEN ALL

*This notebook is intended to be used as a generator of adversarial examples using all five methods.*

## Structure

- Imports
- Global settings
  - Max norm, size of max-norm ball in which I seek for adversarial examples
    - 50 / 255
  - Model that is used
    - MnistCnnPatt
  - batch_size - number of benign examples
    - 10 for testing, 1000 for real application
- Functions that create adversarial examples given benign examples
  - In batch mode
- Generation mechanism and statistics capture
  - save generated adv. examples as images
  - percentage of success of attack

## Requirements

- Simple parallelization
  - meaning that computation can be split across several jupyter notebooks eventually

# 0. Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math

In [3]:
import torch.nn as nn
import torch

In [4]:
from utils import MnistData, Clipper
from models import ModelManager, ModelType
from adversarials import ClassificationAdversarials

In [5]:
import numpy as np
from matplotlib import pyplot as plt

# 1. Gloabal settings

In [6]:
max_norm = 50 / 255

In [7]:
model = ModelManager.get_trained(ModelType.MnistCnnB)

In [8]:
batch_size = 100

In [9]:
data = MnistData(True)

# 2. Functions for adv. examples

### 1. FGSM

In [10]:
def fgsm(model: nn.Module, benign_examples: torch.Tensor, labels: torch.Tensor, max_norm) -> torch.Tensor:
    loss_fn = nn.CrossEntropyLoss()
    benign_examples = benign_examples.detach()
    benign_examples.requires_grad = True
    if benign_examples.grad is not None:
        benign_examples.grad.zero_()
    loss = loss_fn(model(benign_examples), labels)
    loss.backward()
    adversarial_examples = benign_examples + max_norm * benign_examples.grad.apply_(lambda x: 1 if x >= 0 else -1)
    return Clipper.clip_for_image(adversarial_examples)

### 2. I-FGSM

In [11]:
def ifgsm(model: nn.Module, benign_examples: torch.Tensor, labels: torch.Tensor, max_norm) -> torch.Tensor:
    loss_fn = nn.CrossEntropyLoss()
    adversarial_examples = benign_examples.detach()
    step_size = 1e-2
    for _ in range(math.floor(min(max_norm * 4 * 255, max_norm * 255 * 1.25))):
        adversarial_examples.requires_grad = True
        if adversarial_examples.grad:
            adversarial_examples.grad.zero_()
        loss_fn(model(adversarial_examples), labels).backward()
        adversarial_examples = Clipper.clip(
            benign_examples,
            adversarial_examples + step_size * adversarial_examples.grad.apply_(lambda x: 1 if x >= 0 else -1),
            max_norm
        )
    return adversarial_examples

### 3. PGD

In [12]:
def pgd(model: nn.Module, benign_examples: torch.Tensor, labels: torch.Tensor, max_norm) -> torch.Tensor:
    loss_fn = nn.CrossEntropyLoss()
    adversarial_examples = benign_examples.detach() + 2 * max_norm * (torch.rand((len(labels), 1, 28, 28)) - 0.5)
    step_size = 1e-2
    for _ in range(math.floor(min(max_norm * 4 * 255, max_norm * 255 * 1.25))):
        adversarial_examples.requires_grad = True
        if adversarial_examples.grad:
            adversarial_examples.grad.zero_()
        loss_fn(model(adversarial_examples), labels).backward()
        adversarial_examples = Clipper.clip(
            benign_examples,
            adversarial_examples + step_size * adversarial_examples.grad.apply_(lambda x: 1 if x >= 0 else -1),
            max_norm
        )
    return adversarial_examples

### 4. L-BFGS

In [13]:
def mnist_batch_linfty_norm(input:torch.Tensor) -> torch.Tensor:
    return torch.max(torch.max(torch.max(torch.abs(input), dim=3)[0], dim=2)[0], dim=1)[0]

def get_adv(model, benign_examples, target_labels, batch_norm_function, norm_radius, c, clip_always=False):
    step_size = 1e-2
    adversarial_examples = torch.zeros(benign_examples.shape)
    loss_fn = nn.CrossEntropyLoss(reduction='sum')
    for _ in range(100):
        adversarial_examples.requires_grad = True
        if adversarial_examples.grad is not None:
            adversarial_examples.grad.zero_()
        loss = c * batch_norm_function(adversarial_examples - benign_examples).sum() \
            + loss_fn(model(adversarial_examples), target_labels)
        loss.backward()
        adversarial_examples = (adversarial_examples - step_size * adversarial_examples.grad.apply_(lambda x: 1 if x >= 0 else -1)).detach()
        if clip_always:
            adversarial_examples = Clipper.clip_batch(benign_examples, adversarial_examples, batch_norm_function, norm_radius)
    return Clipper.clip_batch(benign_examples, adversarial_examples, batch_norm_function, norm_radius)


def lbfgs_batch(model, benign_examples, labels, norm_radius, batch_norm_function, clip_always=False):
    batch = len(benign_examples)
    all_adversarial_examples = torch.zeros(batch, 9, 28, 28)
    target_labels = torch.tensor([[i for i in range(10) if i != label] for label in labels])
    for i in range(9):
        print(f'Targeted L-BFGS: {i}')
        successful_indexes = []
        unsuccessful_indexes = [i for i in range(batch)]
        c = 100
        while unsuccessful_indexes:
            still_benign_examples = torch.tensor([benign_examples[j].tolist() for j in unsuccessful_indexes])
            still_target_labels = torch.tensor([target_labels[j, i] for j in unsuccessful_indexes])
            adversarial_examples = get_adv(model, still_benign_examples, still_target_labels, batch_norm_function, norm_radius, c, clip_always)
            adversarial_preds = torch.argmax(model(adversarial_examples), dim=1)
            indexes_to_delete = []
            for j in range(len(adversarial_examples)):
                # print(j)
                if adversarial_preds[j] != labels[unsuccessful_indexes[j]] or c <= 0.01:
                    all_adversarial_examples[unsuccessful_indexes[j], i, :, :] = adversarial_examples[j, :, :, :]
                    successful_indexes.append(unsuccessful_indexes[j])
                    indexes_to_delete.append(unsuccessful_indexes[j])
            for j in indexes_to_delete:
                unsuccessful_indexes.remove(j)
            c *= 0.1
    expanded_examples = benign_examples.expand(batch, 9, 28, 28)
    diffs = all_adversarial_examples - expanded_examples
    norms = torch.zeros(batch, 9)
    for i in range(9):
        norms[:, i] = batch_norm_function(diffs[:, i, :, :].reshape(batch, 1, 28, 28))
    selected_adversarial_examples = torch.zeros(benign_examples.shape)
    indexes = torch.argmin(norms, dim=1)
    for i in range(batch):
        selected_adversarial_examples[i, 0, :, :] = all_adversarial_examples[i, indexes[i], :, :]
    return selected_adversarial_examples

In [14]:
# Non-batch code fo L-BFGS targeted attack - too slow

def solve_for_targeted(model, benign_example, target_label, c_lambda, label, max_norm) -> torch.Tensor:
    print(f'\t{target_label}, {c_lambda}')
    step_size = 1e-2
    benign_example = benign_example.unsqueeze(0)
    while len(benign_example.shape) > 4:
        benign_example = benign_example.squeeze(0)
    adv = torch.zeros(benign_example.shape)
    loss_fn = nn.CrossEntropyLoss(reduction='sum')
    norm_of_diff = lambda x, y: torch.max(torch.abs(x - y))
    for _ in range(100):
        adv.requires_grad = True
        if adv.grad is not None:
            adv.grad.zero_()
        loss = c_lambda * norm_of_diff(adv, benign_example) \
            + loss_fn(model(adv), torch.Tensor([target_label]).type(torch.long))
        loss.backward()
        new_adv = Clipper.clip(
            benign_example,
            (adv - step_size * adv.grad.apply_(lambda x: 1 if x >= 0 else -1)),
            max_norm
        )
        adv = new_adv
    if torch.argmax(model(adv), dim=1)[0] == target_label or c_lambda < 1e-5:
        return adv.squeeze(0)
    return None

def lbfgs(model: nn.Module, benign_examples:torch.Tensor, labels: torch.Tensor, max_norm) -> torch.Tensor:
    adversarial_examples = []
    for i in range(len(benign_examples)):
        benign_example, label = benign_examples[i], labels[i]
        advs = []
        for i in [j for j in range(10) if j != label]:
            c_lambda = 1
            adv = None
            while adv is None:
                adv = solve_for_targeted(model, benign_example, i, c_lambda, label, max_norm)
                c_lambda *= 0.7
            advs.append(adv)

        norm_of_diff = lambda x, y: torch.max(torch.abs(x - y))
        norms = []
        for adve in advs:
            norms.append(norm_of_diff(benign_example, adve))
        else:
            minout = min(norms)
            for i in range(len(advs)):
                if norms[i] == minout:
                    adversarial_examples.append(advs[i])
                    break
    return torch.Tensor([example.tolist() for example in adversarial_examples])

### 5. CW

In [15]:
def mnist_batch_linfty_norm(input:torch.Tensor) -> torch.Tensor:
    return torch.max(torch.max(torch.max(torch.abs(input), dim=3)[0], dim=2)[0], dim=1)[0]

def cw_batch(model: nn.Module, benign_examples: torch.Tensor, labels: torch.Tensor, norm_radius, c_lambda: float, batch_norm, clip_always=False) -> torch.Tensor:
    adversarial_examples = torch.zeros(benign_examples.shape)
    loss_fn = nn.CrossEntropyLoss(reduction='sum')
    step_size = 1e-2
    for _ in range(100):
        adversarial_examples.requires_grad = True
        if adversarial_examples.grad is not None:
            adversarial_examples.grad.zero_()
        benign_examples.requires_grad = True
        if benign_examples.grad is not None:
            benign_examples.grad.zero_()
        loss = batch_norm(adversarial_examples - benign_examples).sum() - c_lambda * loss_fn(model(adversarial_examples), labels)
        loss.backward()
        adversarial_examples = (adversarial_examples - step_size * adversarial_examples.grad.apply_(lambda x: 1 if x >= 0 else -1)).detach()
        if clip_always:
            adversarial_examples = Clipper.clip_batch(benign_examples, adversarial_examples, batch_norm, norm_radius)
    return Clipper.clip_batch(benign_examples, adversarial_examples, batch_norm, norm_radius)

In [16]:
# Non-batch code for CW attack - too slow

def solve_for(model, benign_image, label, c_lambda, max_norm):
    step_size = 1e-2
    adv = torch.zeros(benign_image.shape)
    loss_fn = nn.CrossEntropyLoss(reduction='sum')
    norm_of_diff = lambda x, y: torch.max(torch.abs(x - y))
    adv = adv.unsqueeze(0)
    benign_image = benign_image.unsqueeze(0)
    for _ in range(100):
        adv.requires_grad = True
        if adv.grad is not None:
            adv.grad.zero_()
        loss = norm_of_diff(adv, benign_image) \
            - c_lambda * loss_fn(model(adv), torch.Tensor([label]).type(torch.long))
        loss.backward()
        new_adv = Clipper.clip(
            benign_image,
            (adv - step_size * adv.grad.apply_(lambda x: 1 if x >= 0 else -1)),
            max_norm
        )
        adv = new_adv
    if torch.argmax(model(adv), dim=1)[0] != label or c_lambda > 10:
        return adv.squeeze(0)
    return None

def cw(model: nn.Module, benign_examples: torch.Tensor, labels: torch.Tensor, max_norm) -> torch.Tensor:
    advs = []
    for i in range(len(benign_examples)):
        print(f'--- {i} ---')
        benign_example, label = benign_examples[i], labels[i]
        adv = None
        c_lambda = 1e-2
        while adv is None:
            adv = solve_for(model, benign_example, label, c_lambda, max_norm)
            c_lambda *= 1.1
        advs.append(adv)
    return torch.Tensor([adv.tolist() for adv in advs])

# 3. Generation mechanism

In [17]:
benign_examples, labels = data.choose_first_well_classified(batch_size, model)

In [18]:

# Generation
print('Starting FGSM attack')
fgsm_examples = fgsm(model, benign_examples, labels, max_norm)

print('Starting I-FGSM attack')
ifgsm_examples = ifgsm(model, benign_examples, labels, max_norm)

print('Starting PGD attack')
pgd_examples = pgd(model, benign_examples, labels, max_norm)

print('Starting targeted L-BFGS attack')
lbfgs_examples = lbfgs_batch(model, benign_examples, labels, max_norm, mnist_batch_linfty_norm, True)

print('Starting CW attack with c_lambda = 1')
cw_examples = cw_batch(model, benign_examples, labels, max_norm, 1, mnist_batch_linfty_norm, True)

Starting FGSM attack
Starting I-FGSM attack
Starting PGD attack
Starting targeted L-BFGS attack
Targeted L-BFGS: 0


In [ ]:

# Save'em all
for i in range(batch_size):
    example = np.array(benign_examples[i].detach()).reshape(28, 28)
    plt.imshow(example, cmap='gray', vmin=0, vmax=1)
    plt.axis("off")
    plt.savefig(f"AEXAMPLES\\ALLGEN\\benign_{i}.png", bbox_inches="tight", pad_inches=0)
    
    example = np.array(fgsm_examples[i].detach()).reshape(28, 28)
    plt.imshow(example, cmap='gray', vmin=0, vmax=1)
    plt.axis("off")
    plt.savefig(f"AEXAMPLES\\ALLGEN\\fgsm_{i}.png", bbox_inches="tight", pad_inches=0)
    
    example = np.array(ifgsm_examples[i].detach()).reshape(28, 28)
    plt.imshow(example, cmap='gray', vmin=0, vmax=1)
    plt.axis("off")
    plt.savefig(f"AEXAMPLES\\ALLGEN\\ifgsm_{i}.png", bbox_inches="tight", pad_inches=0)
    
    example = np.array(pgd_examples[i].detach()).reshape(28, 28)
    plt.imshow(example, cmap='gray', vmin=0, vmax=1)
    plt.axis("off")
    plt.savefig(f"AEXAMPLES\\ALLGEN\\pgd_{i}.png", bbox_inches="tight", pad_inches=0)
    
    example = np.array(lbfgs_examples[i].detach()).reshape(28, 28)
    plt.imshow(example, cmap='gray', vmin=0, vmax=1)
    plt.axis("off")
    plt.savefig(f"AEXAMPLES\\ALLGEN\\lbfgs_{i}.png", bbox_inches="tight", pad_inches=0)
    
    example = np.array(cw_examples[i].detach()).reshape(28, 28)
    plt.imshow(example, cmap='gray', vmin=0, vmax=1)
    plt.axis("off")
    plt.savefig(f"AEXAMPLES\\ALLGEN\\cw_{i}.png", bbox_inches="tight", pad_inches=0)

In [ ]:

fgsm_adversarials = ClassificationAdversarials.get_adversarials(model, benign_examples, labels, fgsm_examples)
print(f'fgsm: {len(fgsm_adversarials)}')

ifgsm_adversarials = ClassificationAdversarials.get_adversarials(model, benign_examples, labels, ifgsm_examples)
print(f'ifgsm: {len(ifgsm_adversarials)}')

pgd_adversarials = ClassificationAdversarials.get_adversarials(model, benign_examples, labels, pgd_examples)
print(f'pgd: {len(pgd_adversarials)}')

lbfgs_adversarials = ClassificationAdversarials.get_adversarials(model, benign_examples, labels, lbfgs_examples)
print(f'lbfgs: {len(lbfgs_adversarials)}')

cw_adversarials = ClassificationAdversarials.get_adversarials(model, benign_examples, labels, cw_examples)
print(f'cw: {len(cw_adversarials)}')    

# 4. Ad Hoc